In [15]:
from __future__ import print_function
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.stats import zscore
import seaborn as sns
import sys,os
from mapper import expand, parse_mapping_table, apply_mappers
%matplotlib  inline

In [2]:
gene_id = "ENTREZID"
raw_data_dir = "/home/olya/SFU/Hossein/PDX/" 
preprocessed_data_dir =  "/home/olya/SFU/Hossein/v2/preprocessed/exprs/"
root_dir = "/home/olya/SFU/Hossein/v2/"
# wget https://media.nature.com/original/nature-assets/nm/journal/v21/n11/extref/nm.3954-S2.xlsx
# download Entrez ID mapping file

# Data source references

In [14]:
from IPython.display import HTML, display
import tabulate
table = [["dataset","Pubmed IDs","Reference"],
         ["TCGA",
          "TCGA Research Network publications, e.g. PMID:24071849,https://doi.org/10.7908/c11g0km9",
          "http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/"],
         ["PDX","PMID:26479923","https://media.nature.com/original/nature-assets/nm/journal/v21/n11/extref/nm.3954-S2.xlsx"]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

0,dataset,Pubmed IDs,Reference
1,TCGA,"TCGA Research Network publications, e.g. PMID:24071849,https://doi.org/10.7908/c11g0km9",http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/
2,PDX,PMID:26479923,https://media.nature.com/original/nature-assets/nm/journal/v21/n11/extref/nm.3954-S2.xlsx


# PDX 

- for 22665 Gene symbols are converted to Gene IDs 
- gene expression profiles (FPKM) for 399 samples converted to log2(TPM+1)

In [3]:
exprs  = pd.read_excel(raw_data_dir+"nm.3954-S2.xlsx","RNAseq_fpkm")
exprs.set_index("Sample",inplace=True,drop=True)
print(exprs.shape)
exprs.head() 

(22665, 399)


,X-1004,X-1008,X-1027,X-1095,X-1119,X-1156,X-1167,X-1169,X-1172,X-1173,...,X-5713,X-5717,X-5727,X-5739,X-5808,X-5959,X-5974,X-5975,X-6030,X-6047
Sample,,,,,,,,,,,,,,,,,,,,,
A1BG,2.75,8.97,0.14,0.13,0.08,17.52,0.18,0.41,0.08,0.00,...,21.24,0.46,18.70,0.00,8.30,14.41,5.09,6.35,0.09,3.22
A1BG-AS1,2.48,3.25,0.00,0.00,0.00,6.52,0.00,0.00,0.00,0.00,...,15.21,0.09,3.42,0.00,8.30,8.36,3.71,4.52,0.00,3.24
A1CF,0.02,0.03,1.30,2.83,2.87,0.72,3.41,0.01,1.84,2.96,...,0.01,0.45,0.00,0.01,0.00,0.00,0.02,0.03,4.63,0.02
A2LD1,4.87,0.81,6.45,4.94,11.07,0.87,3.28,0.32,0.61,7.10,...,1.33,2.51,2.67,3.13,0.44,2.96,0.00,1.99,2.57,1.50
A2M,0.01,71.17,0.00,3.69,0.00,58.16,0.00,0.02,94.12,0.02,...,463.32,0.04,33.35,0.00,254.97,41.52,2.32,0.00,0.00,0.16


# Mapping of gene symbols to EntrezID using current gene_info file prowided by NCBI:


 * Download the unzip the file  
\# wget ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
\# gunzip Homo_sapiens.gene_info.gz;
 * Specify *hgnc_file* variable in this notebook
 * Mapping strategy
 
1). Unknown genes and genes belonging to organisms other than H.sapiens were excluded.

2). First each query symbol was tried to match with any of current "Symbol" directly. If the query symbol mapped to one of current symbol but has no Gene ID, the query symbol was marked as not mapped. 

3). If the query symbol matches none of current symbols, we tried to match it with one of "Synonyms". Genes matched no synonym as well as matched ambiguous synonyms correponding more than one Gene ID were condiered not mapped. 

4). At this point, many of not recognized symbols had LOCXXXXXXXXX format. 
According to the documntation provided by NCBI: "Symbols beginning with LOC. When a published symbol is not available, and orthologs have not yet been determined, Gene will provide a symbol that is constructed as 'LOC' + the GeneID. This is not retained when a replacement symbol has been identified, although queries by the LOC term are still supported. In other words, a record with the symbol LOC12345 is equivalent to GeneID = 12345. So if the symbol changes, the record can still be retrieved on the web using LOC12345 as a query, or from any file using GeneID = 12345" e.g. :

    - LOC100093631 -> 100093631 
    - LOC100129726 -> 100129726
    - etc. 

Therefore all genes started with LOC were converted to Gene IDs removing "LOC" from query term. If resulting Gene ID matched none of current Gene IDs the symbol was considered not mapped. 

4). Several pairs of query gene symbols matched current symbol and synonym of the same Entrez gene ID, e.g. AGAP8 and AGAP4, ANXA8L1 and ANXA8L2, etc.
Expressions of these genes were summarized because such genes were merged to a single gene in newer assembly versions.

In [5]:
hgnc_file = root_dir+"HGNC_5.10.2018.txt"

hgnc = pd.read_csv(hgnc_file, sep ="\t",index_col=0)#
#print(hgnc.shape, len(set(hgnc.index.values)))
approved = hgnc.loc[hgnc["Status"] == "Approved",:]
hgnc_prev =  expand(approved[["Previous Symbols","Entrez Gene ID"]],column="Previous Symbols",sep=", ") 
hgnc_prev = parse_mapping_table(hgnc_prev, "Previous Symbols","Entrez Gene ID")

2547 rows with both Previous Symbols and Entrez Gene ID empty
Ok: no duplicated pairs detected
27336 rows with empty Previous Symbols were excluded
173 Previous Symbols ids mapped to no Entrez Gene ID
83 Previous Symbols mapped to multiple Entrez Gene ID
4697 different Previous Symbols mapped to the same Entrez Gene ID
9336 Previous Symbols can be mapped directly to Entrez Gene ID


In [6]:
hgnc_syn =  expand(approved[["Synonyms","Entrez Gene ID"]],column="Synonyms",sep=", ") 
hgnc_syn = parse_mapping_table(hgnc_syn, "Synonyms","Entrez Gene ID")

2224 rows with both Synonyms and Entrez Gene ID empty
Ok: no duplicated pairs detected
18283 rows with empty Synonyms were excluded
662 Synonyms ids mapped to no Entrez Gene ID
962 Synonyms mapped to multiple Entrez Gene ID
27509 different Synonyms mapped to the same Entrez Gene ID
10612 Synonyms can be mapped directly to Entrez Gene ID


In [7]:
NCBI = pd.read_csv(root_dir+"Homo_sapiens.gene_info",sep = "\t")
NCBI = NCBI[["#tax_id","GeneID","Symbol","Synonyms","type_of_gene"]]
NCBI = NCBI.loc[NCBI["#tax_id"] == 9606]
NCBI = NCBI.loc[NCBI["type_of_gene"] != "unknown"]
ncbi_symbols = parse_mapping_table(NCBI, "Symbol","GeneID")

Ok: no empty rows detected
Ok: no duplicated pairs detected
Ok: All Symbol rows are not empty.
Ok: All Symbol are mapped to GeneID
16 Symbol mapped to multiple GeneID
Ok: All GeneID are unique
59266 Symbol can be mapped directly to GeneID


In [8]:
ncbi_synonyms = expand(NCBI[["Synonyms","GeneID"]],column="Synonyms",sep="|") 
ncbi_synonyms = parse_mapping_table(ncbi_synonyms, "Synonyms","GeneID")

Ok: no empty rows detected
Ok: no duplicated pairs detected
Ok: All Synonyms rows are not empty.
Ok: All Synonyms are mapped to GeneID
3145 Synonyms mapped to multiple GeneID
49179 different Synonyms mapped to the same GeneID
10839 Synonyms can be mapped directly to GeneID


In [9]:
exprs = apply_mappers(exprs, ncbi_symbols, ncbi_synonyms, verbose = True,handle_duplicates = "sum")
exprs.head(5)

Mapped: 22467 
	directly via main_mapper 20272 
	via alternative mapper 471 
	via one of multiple synonyms in alternative mapper 1026 
	LOC 698 
Unmapped: 198 
	recognized symbols without Entrez ID 0 
	multiple query_ids map to the same target_id 0 
	query_ids map to multiple target_ids in the main mapper 0 
	query_ids map to multiple target_ids in the alternative mapper 54 
	LOC not found in Entrez 26 
	Not found at all: 118


AttributeError: 'tuple' object has no attribute 'head'

### FPKM to TPM conversion
Let $X_i$ is a number of fragments mapped to a transcript, $N$ is a total number of fragments sequensed (and assigned to any transcript) and $\widetilde{l_i}$  is an effective length of a transcript (i.e. how many fragments with average length $\mu_{frag}$ can generate a transcript of length $l_i$: $\widetilde{l_i} = l_i - \mu_{frag}+1$)

*FPKM* - fragments per kilobase of exon (i.e. effective length) per million reads mapped

$FPKM_i = \frac{X_i}{\frac{$\widetilde{l_i}}{10^3}*\frac{N}{10^6}} = \frac{X_i}{l_iN}*10^9 $

*TPM* - transcripts per million of transcripts.

In turn, $\frac{X_i}{\widetilde{l_i}}$ is estimated number of transcripts 

$TPM_i = \frac{\frac{X_i}{\widetilde{l_i}}}{\sum_j{\frac{X_j}{\widetilde{l_j}}}}*10^6 $


### how to convert FPKM to TPM
Divide both numerator and denominator by $N$ and mutiply by $10^9$:

$TPM_i = \frac{\frac{X_i}{N\widetilde{l_i}}*10^9}{\sum_j{\frac{X_j}{N\widetilde{l_j}}*10^9}}*10^6 = \frac{FPKM_i}{\sum_j{FPKM_j}} * 10^6$

Sources:
 - What the FPKM? https://haroldpimentel.wordpress.com/2014/05/08/what-the-fpkm-a-review-rna-seq-expression-units/
 - https://www.biostars.org/p/160989/
 - Lior Pachter https://arxiv.org/pdf/1104.3889.pdf


In [ ]:
##  FPKM convert to log2(TPM+1)
sum_fpkm = exprs.apply(sum,axis=0)
sum_fpkm.head()

In [ ]:
tpm = exprs / sum_fpkm *1000000 +1
tpm.head()

In [ ]:

tpm = tpm.applymap(np.log2)
#tpm.to_csv(preprocessed_data_dir + "/PDX.FPKM2TPMplus1log2.Expr.tsv",sep="\t")
print(tpm.shape)
tpm.head()


# TCGA 

* from http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/ download RSEM files, "scaled estimate" per gene. 

* RSEM Scaled estimate is an aboundance of a transcript divided by sum of aboundance over all the transcripts.  Therefore $TPM_i=ScaledEstimate_i*10^6$

* Resulted TPM were log2-transformed

In [3]:
# replace with downloading 
tcga_tmp_dir = "/home/olya/SFU/Hossein/TCGA/expression__2016_01_28/"

In [10]:
f_ext = ".rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.data.txt"
for fpath in os.listdir(tcga_tmp_dir):
    if fpath.startswith("gdac.broadinstitute.org") and not fpath.endswith(".tar.gz") :
        cohort = fpath.split("_")[1].replace(".Merge","")
        #print(fpath, cohort)
        fname = cohort + f_ext
        try:
            exprs = pd.read_csv(tcga_tmp_dir+"/"+fpath+"/"+fname,sep="\t",index_col=0)
            # drop "gene_id" and keep only "scaled_estimate" columns
            exprs = exprs.loc[:,exprs.T.loc[exprs.T["gene_id"]=="scaled_estimate",:].index]
            exprs = exprs.iloc[1:,]
            exprs.rename(index = lambda x :  int(x.split("|")[1]),
                         columns = lambda x : x.replace(".1",""),inplace = True)
            exprs.index.name = "ENTREZID"
            # convert scaled_extimates to log2(TPM+1)
            exprs = exprs.applymap(lambda x : np.log2(float(x)*1000000+1))
            exprs = exprs.sort_index()
            exprs.to_csv(preprocessed_data_dir +"TCGA-"+cohort+"_exprs.RSEMscaled_est2TPMplus1log2.tsv",sep  ="\t")
            print(cohort,exprs.shape)
        except:
            print(cohort,"No expression data.")


    

gdac.broadinstitute.org_BRCA.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.Level_3.2016012800.0.0 BRCA


/home/olya/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,355,356,358,359,

BRCA (20531, 1212)
gdac.broadinstitute.org_STES.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.Level_3.2016012800.0.0 STES


/home/olya/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,355,356,358,359,

STES (20531, 646)
gdac.broadinstitute.org_SKCM.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.Level_3.2016012800.0.0 SKCM


...

STAD (20531, 450)
gdac.broadinstitute.org_OV.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.Level_3.2016012800.0.0 OV


/home/olya/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,355,356,358,359,

OV (20531, 307)
gdac.broadinstitute.org_BLCA.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.Level_3.2016012800.0.0 BLCA


/home/olya/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,355,356,358,359,

BLCA (20531, 427)
